In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy.linalg import eigh # for finding the eigenvalue and eigenvector

In [ ]:
data_path = 'Data/arr_0.npy'
data_raw = np.load(data_path)
data_raw.shape

In [ ]:
def ComputeMean(data):
    """Input : n dimensional numpy array
        Output : mean"""
    return np.sum(data,axis=0)/len(data)

def Covariance(data):
    """Input: numpy array of n-dim"""
    N, M = data.shape
    cov = np.zeros((M, M))
    for i in range(M):
        mean_i = np.sum(data[:, i]) / N
        for j in range(M):
            mean_j = np.sum(data[:, j]) / N
            cov[i, j] = np.sum((data[:, i] - mean_i) * (data[:, j] - mean_j)) / (N - 1)
    return cov  

def ComputeStd(data):
    return np.std(data)

In [ ]:
"""Mean Centering of the data"""

# Finding the mean of data

data_mean = ComputeMean(data_raw)

# Mean centering of the data

data_centered = data_raw - data_mean

std = ComputeStd(data_raw)

print(std)

data_standard = data_centered/std

In [ ]:
num_rows = 8
num_cols = 8

# Create a subplot with the appropriate number of rows and columns
fig, axes = plt.subplots(num_rows, num_cols, figsize=(8, 8))

# Iterate through the data and plot each image
for i in range(num_rows):
    for j in range(num_cols):
        # Reshape the 64-length feature vector into an 8x8 matrix
        image = data_centered[i * num_cols + j].reshape(8, 8)

        # Display the image in the current subplot
        axes[i, j].imshow(image, cmap='gray')
        axes[i, j].axis('off')

# Adjust spacing between subplots
plt.subplots_adjust(wspace=0.1, hspace=0.1)
# plt.title('Visualization of the data', fontsize=22)
plt.savefig('./plots/data_centered')

# Show the plot
plt.show()

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(data_centered)

# Step 3: Calculate the cumulative explained variance ratio
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)

# Step 4: Plot the cumulative explained variance
plt.plot(cumulative_variance, marker='o')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Variance Explained')
plt.title('Cumulative Variance Explained by Principal Components')
plt.grid()

# Step 5: Determine the number of components that contribute to 90% of the variance
n_components_90 = np.argmax(cumulative_variance >= 0.90) + 1
print(f"Number of components contributing to 90% of variance: {n_components_90}")

plt.axvline(x=n_components_90, color='r', linestyle='--')
plt.axhline(y=0.90, color='r', linestyle='--')
plt.show()

In [ ]:
"""Finding the covariance matrix of the centered data"""

data_centered_cov = Covariance(data_centered)
data_standard_cov = Covariance(data_standard)


In [ ]:
"""Finding the eigenvalue and eigenvector of the covariance matrix"""
eigenvalue , eigenvector =  eigh(data_centered_cov)
eigenvalue_std , eigenvector_std =  eigh(data_standard_cov)


In [ ]:
eigenvector.shape

In [ ]:
sorted_index = np.argsort(eigenvalue)[::-1]
sorted_eigenvalue = eigenvalue[sorted_index]
sorted_eigenvectors = eigenvector[:,sorted_index]


In [ ]:
sorted_index_std = np.argsort(eigenvalue_std)[::-1]
sorted_eigenvalue_std = eigenvalue_std[sorted_index_std]
sorted_eigenvectors_std = eigenvector_std[:,sorted_index_std]


In [ ]:
total_variance = np.sum(sorted_eigenvalue)
explained_variance_ratio = sorted_eigenvalue / total_variance
print('explained_variance_ratio',explained_variance_ratio)
cumulative_variance = np.cumsum(explained_variance_ratio)

# Step 6: Plot the cumulative explained variance
plt.plot(cumulative_variance, marker='o')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Variance Explained')
plt.title('Cumulative Variance Explained by Principal Components')
plt.grid()

# Step 7: Determine the number of components contributing to 90% of the variance
n_components_90 = np.argmax(cumulative_variance >= 0.90) + 1
print(f"Number of components contributing to 90% of variance: {n_components_90}")

plt.axvline(x=n_components_90, color='r', linestyle='--')
plt.axhline(y=0.90, color='r', linestyle='--')
plt.savefig('./plots/Cumulative Variance Explained by Principal Components')
plt.show()

In [ ]:
total_variance_std = np.sum(sorted_eigenvalue_std)
explained_variance_ratio_std = sorted_eigenvalue_std / total_variance_std
print('explained_variance_ratio_std',explained_variance_ratio_std)
cumulative_variance_std = np.cumsum(explained_variance_ratio_std)

# Step 6: Plot the cumulative explained variance
plt.plot(cumulative_variance_std, marker='o')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Variance Explained')
plt.title('Cumulative Variance Explained by Principal Components')
plt.grid()

# Step 7: Determine the number of components contributing to 90% of the variance
n_components_90_std = np.argmax(cumulative_variance_std >= 0.90) + 1
print(f"Number of components contributing to 90% of variance: {n_components_90_std}")

plt.axvline(x=n_components_90_std, color='r', linestyle='--')
plt.axhline(y=0.90, color='r', linestyle='--')
plt.savefig('./plots/Cumulative Variance Explained by Principal Components')
plt.show()

In [ ]:
n_features = 64
eig_vals_total = sum(eigenvalue)
explained_variance = [(i / eig_vals_total)*100 for i in sorted_eigenvalue]
explained_variance = np.round(explained_variance, 2)
cum_explained_variance = np.cumsum(explained_variance)

print('Explained variance: {}'.format(explained_variance))
print('Cumulative explained variance: {}'.format(cum_explained_variance))

plt.plot(np.arange(1,n_features+1), cum_explained_variance, '-o')
plt.xticks(np.arange(1,n_features+1))
plt.xlabel('Number of components')
plt.ylabel('Cumulative explained variance');
plt.show()

In [ ]:
"""Reconstruction of the data with the 21 eigenvecctors"""

n_PCA_components = 64
selected_eigenvectors = sorted_eigenvectors[:, :n_PCA_components]
reduced_data = np.dot(data_centered, selected_eigenvectors)
reconstructed_data = np.dot(reduced_data, selected_eigenvectors.T)
reconstructed_data = (reconstructed_data) + data_mean


num_rows = 8
num_cols = 8

# Create a subplot with the appropriate number of rows and columns
fig, axes = plt.subplots(num_rows, num_cols, figsize=(8, 8))

# Iterate through the data and plot each reconstructed image
for i in range(num_rows):
    for j in range(num_cols):
        # Reshape the 64-length feature vector into an 8x8 matrix for both original and reconstructed data
        # original_image = data_raw[i * num_cols + j].reshape(8, 8)
        reconstructed_image = reconstructed_data[i * num_cols + j].reshape(8, 8)

        # Display the original and reconstructed images side by side in the current subplot
        # axes[i, j].imshow(original_image, cmap='gray', aspect='auto')
        axes[i, j].imshow(reconstructed_image, cmap='viridis', alpha=0.7, aspect='auto')
        axes[i, j].axis('off')

# Adjust spacing between subplots
plt.subplots_adjust(wspace=0.1, hspace=0.1)
plt.savefig(f'./plots/Reconstruction of the data with the {n_PCA_components} PCA Components')
# Show the plot
plt.show()

In [ ]:
"""Reconstruction of the data with the 21 eigenvecctors"""

n_PCA_components = 64
selected_eigenvectors_std = sorted_eigenvectors_std[:, :n_PCA_components]
reduced_data_std = np.dot(data_standard, selected_eigenvectors_std)
reconstructed_data_std = np.dot(reduced_data_std, selected_eigenvectors_std.T)
reconstructed_data_std = (reconstructed_data_std * std) + data_mean


num_rows = 8
num_cols = 8

# Create a subplot with the appropriate number of rows and columns
fig, axes = plt.subplots(num_rows, num_cols, figsize=(8, 8))

# Iterate through the data and plot each reconstructed image
for i in range(num_rows):
    for j in range(num_cols):
        # Reshape the 64-length feature vector into an 8x8 matrix for both original and reconstructed data
        # original_image = data_raw[i * num_cols + j].reshape(8, 8)
        reconstructed_image = reconstructed_data_std[i * num_cols + j].reshape(8, 8)

        # Display the original and reconstructed images side by side in the current subplot
        # axes[i, j].imshow(original_image, cmap='gray', aspect='auto')
        axes[i, j].imshow(reconstructed_image, cmap='viridis', alpha=0.7, aspect='auto')
        axes[i, j].axis('off')

# Adjust spacing between subplots
plt.subplots_adjust(wspace=0.1, hspace=0.1)
plt.savefig(f'./plots/Reconstruction of the data with the {n_PCA_components} PCA Components(standardised)')
# Show the plot
plt.show()

In [ ]:
dimensions = [2, 4, 8, 16,21,64]

# Dictionary to store the MSE values for each dimension
mse_values = {}

# Calculate the MSE for each dimension
for n_components in dimensions:
    # Select the first 'n_components' eigenvectors
    selected_eigenvectors = sorted_eigenvectors[:, :n_components]
    
    # Project the data onto the selected eigenvectors to obtain the reduced representation
    reduced_data = np.dot(data_centered, selected_eigenvectors)
    
    # Reconstruct the data from the reduced representation
    reconstructed_data = np.dot(reduced_data, selected_eigenvectors.T)
    
    # De-standardize the reconstructed data
    reconstructed_data = (reconstructed_data) + data_mean
    
    # Calculate the Mean Square Error (MSE)
    mse = np.mean(np.square(data_raw - reconstructed_data))
    
    # Store the MSE value for this dimension
    mse_values[n_components] = mse

# Print and interpret the MSE values
for n_components, mse in mse_values.items():
    print(f"Dimension {n_components}: MSE = {mse:.4f}")

# Interpret the optimal dimension based on the MSE values
optimal_dimension = min(mse_values, key=mse_values.get)
print(f"Optimal dimension based on MSE: {optimal_dimension}")

In [ ]:
#error in terms of eigenvector = avg of remaining eigenvectors

# To DO

In [ ]:
k = 64
W = sorted_eigenvectors[:k, :] # Projection matrix

print(W.shape)

data_projected = data_centered.dot(W.T)

print(data_projected.shape)

In [ ]:
num_rows = 8
num_cols = 8

# Create a subplot with the appropriate number of rows and columns
fig, axes = plt.subplots(num_rows, num_cols, figsize=(8, 8))

# Iterate through the data and plot each image
for i in range(num_rows):
    for j in range(num_cols):
        # Reshape the 64-length feature vector into an 8x8 matrix
        image = data_projected[i * num_cols + j].reshape(8,8)

        # Display the image in the current subplot
        axes[i, j].imshow(image, cmap='gray')
        axes[i, j].axis('off')

# Adjust spacing between subplots
plt.subplots_adjust(wspace=0.1, hspace=0.1)
# plt.title('Visualization of the data', fontsize=22)


# Show the plot
plt.show()

In [ ]:
fig = plt.figure(figsize=(6, 6))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)
for i in range(20):
    ax = fig.add_subplot(5, 5, i + 1, xticks=[], yticks=[])
    ax.imshow(data_raw[i], cmap=plt.cm.binary, interpolation='nearest')
    # label the image with the target value
    # ax.text(0, 7, str(y_train[i]))
plt.show()

In [ ]:
total_egnvalues = sum(sorted_eigenvalue)
var_exp = [(i/total_egnvalues) for i in sorted(sorted_eigenvalue, reverse=True)]

var_exp = np.array(var_exp)
variance_exp_cumsum = var_exp.cumsum().round(2)
fig, axes = plt.subplots(1,1,figsize=(16,7), dpi=100)
plt.plot(var_exp, color='firebrick')
plt.title('Screeplot of Variance Explained %', fontsize=22)
plt.xlabel('Number of Principal Components', fontsize=16)
# path = '/home/tenet/prml_assignment/variance_max.png'
# plt.savefig(path)
plt.show()


In [ ]:
cov_mat = np.cov(a, rowvar = False)
cov_mat

In [ ]:
cov_mat-x

In [25]:
data_Q3 = np.array([
    [ 5.51, 5.35 ,3.5],
    [ 20.82, 24.03 ,3.5],
    [ -0.77, -0.57 ,3.5],
    [ 19.30, 19.38 ,3.5],
    [ 14.24, 12.77 ,3.5],
    [ 9.74, 9.68 ,3.5],
    [ 11.59, 12.06 ,3.5],
    [ -6.08, -5.22 ,3.5]
])

data_Q3_mean = ComputeMean(data_Q3)
d_=data_Q3-data_Q3_mean
data_Q3_cov_ = Covariance(d_)

eigenvalue , eigenvector =  eigh(data_Q3_cov_)
print('eigenvalue',eigenvalue)

print('eigenvector',eigenvector)
data_Q3_cov_
d_

eigenvalue [  0.81663462 181.45636359]
eigenvector [[-0.71990351  0.69407416]
 [ 0.69407416  0.71990351]]


array([[-3.783750e+00, -4.335000e+00],
       [ 1.152625e+01,  1.434500e+01],
       [-1.006375e+01, -1.025500e+01],
       [ 1.000625e+01,  9.695000e+00],
       [ 4.946250e+00,  3.085000e+00],
       [ 4.462500e-01, -5.000000e-03],
       [ 2.296250e+00,  2.375000e+00],
       [-1.537375e+01, -1.490500e+01]])

In [ ]:
a = np.array([[20.82],[24.03]])
b = np.array([[-.694,-0.72]])
x = np.matmul([[20.82],[24.03]],[[-.694,-0.72]])
b.shape
x

In [ ]:
import matplotlib.pyplot as plt 

x = np.array([     5.51,
     20.82,
     -0.77,
     19.30,
     14.24,
     9.74,
     11.59,
     -6.08])
y = np.array([     5.51,
      24.03,
      -0.57,
      19.38,
      12.77,
      9.68,
      12.06,
      -5.22])
plt.scatter(x,y)